In [65]:
import pandas as pd
import os
import time

path_file = "sales_data.csv"
use_cols = ['customer_id', 'product_id', 'quantity', 'price']

dtypes = {          
    'quantity': 'int16',          
    'price': 'float32',          
    'region': 'category'          
}

chunk_size = 1000
data = pd.DataFrame() 

def check_fun_error(fun):
    def wrapper(*args, **kwargs):
        try:
            return fun(*args, **kwargs)  
        except Exception as e: 
            print(f"❌ An error occurred in function '{fun.__name__}': {str(e)}")
            exit(1)  
    return wrapper


def timing(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Temps d'exécution de {func.__name__}: {end_time - start_time:.4f} secondes")
        return result
    return wrapper
            
                 
@check_fun_error  
def load_file_with_chunks(path_file, chunk_size) -> None:
    global data 
    try : 
        for ch in pd.read_csv(path_file, usecols= use_cols, dtype= dtypes  , chunksize=chunk_size):
            data = pd.concat([data, ch], ignore_index=True).sample(frac=0.1  , random_state = 2)
    finally : 
        if len(data) > 0:
            print("✅ Data loaded successfully.")
        else:
            print("⚠️ Something went wrong, data is empty.")


@check_fun_error
@timing
def convert_to_feather_or_parquet(type_convert) : 
    global data
    if type_convert.lower() == "feather":
        print("You convert to feather ")
        return data.to_feather('data.feather')
    if type_convert.lower() == "parquet" :
        print("You convert to parquet  ") 
        return data.to_parquet('data.parquet')
    
@check_fun_error
@timing
def read_files_feather_parquet_csv(type_file):
    type_file = type_file.lower()  
    if type_file == "feather":
        print("You are reading a Feather file")
        return pd.read_feather("data.feather")
    elif type_file == "parquet":
        print("You are reading a Parquet file")
        return pd.read_parquet("data.parquet")
    elif type_file == "csv":
        print("You are reading a CSV file")
        return pd.read_csv("sales_data.csv")
    else:
        print("Unsupported file type. Please choose from 'feather', 'parquet', or 'csv'.")
        return None       

@check_fun_error
def compare_size_files():
    # Obtenir la taille des fichiers en octets
    file_size_csv = os.path.getsize('sales_data.csv')
    file_size_feather = os.path.getsize('data.feather')
    file_size_parquet = os.path.getsize('data.parquet')
    
    # Afficher les tailles des fichiers
    print(f"Taille du fichier CSV: {file_size_csv:.2f} octets")
    print(f"Taille du fichier Feather: {file_size_feather:.2f} octets")
    print(f"Taille du fichier Parquet: {file_size_parquet:.2f} octets")

    # Comparaison des tailles pour déterminer le fichier le plus volumineux
    if file_size_csv > file_size_feather and file_size_csv > file_size_parquet:
        print("Le fichier CSV est le plus volumineux.")
    elif file_size_feather > file_size_csv and file_size_feather > file_size_parquet:
        print("Le fichier Feather est le plus volumineux.")
    elif file_size_parquet > file_size_csv and file_size_parquet > file_size_feather:
        print("Le fichier Parquet est le plus volumineux.")
    else:
        print("Plusieurs fichiers ont des tailles équivalentes.")
            
        
if len(data)  ==  0 :
    load_file_with_chunks(path_file=path_file, chunk_size=chunk_size)

data_transaction_supp_100 = data[ data.price > 100 ]
    
print("#" * 60 )
convert_to_feather_or_parquet("feather")

print("#" * 60 )
convert_to_feather_or_parquet("parquet")

print("#" * 60 )
# df = read_files_feather_parquet_csv("feather")

print("#" * 60 )
# df = read_files_feather_parquet_csv("parquet")
print("#" * 60 )
# df = read_files_feather_parquet_csv("csv")

# print(df.head(10))

compare_size_files()

print(data.info())
print(data_transaction_supp_100.head(10))



✅ Data loaded successfully.
############################################################
You convert to feather 
Temps d'exécution de convert_to_feather_or_parquet: 0.0042 secondes
############################################################
You convert to parquet  
Temps d'exécution de convert_to_feather_or_parquet: 0.0043 secondes
############################################################
############################################################
############################################################
Taille du fichier CSV: 57359485.00 octets
Taille du fichier Feather: 6282.00 octets
Taille du fichier Parquet: 6152.00 octets
Le fichier CSV est le plus volumineux.
<class 'pandas.core.frame.DataFrame'>
Index: 111 entries, 959 to 230
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   customer_id  111 non-null    object 
 1   product_id   111 non-null    object 
 2   quantity     111 non-null    int16  
 3   p

In [ ]:
import h5py
import pandas as pd
import numpy as np


data = {
    'TransactionID': [1, 2, 3, 4, 5],
    'Produit': ['Produit A', 'Produit B', 'Produit C', 'Produit D', 'Produit E'],
    'Prix': [50, 150, 80, 120, 200],
    'Quantité': [1, 2, 1, 3, 1]
}
normal_data = pd.DataFrame(data)

# Convertir les types avec numpy
normal_data['TransactionID'] = normal_data['TransactionID'].astype(np.int32)
normal_data['Produit'] = normal_data['Produit'].astype(np.str_)  
normal_data['Prix'] = normal_data['Prix'].astype(np.float64)
normal_data['Quantité'] = normal_data['Quantité'].astype(np.int32)

# Filtrer les transactions dont le prix est supérieur à 100
data_transaction_supp_100 = normal_data[normal_data['Prix'] > 100]

# Convertir les DataFrame en un format structuré numpy pour HDF5
dtype = np.dtype([('TransactionID', np.int32),
                  ('Produit', np.string_, 100),  # Convertir les chaînes en np.string_ de longueur fixe
                  ('Prix', np.float64),
                  ('Quantité', np.int32)])

# Convertir en tableau structuré
normal_data_array = np.array(list(normal_data.itertuples(index=False, name=None)), dtype=dtype)
data_transaction_supp_100_array = np.array(list(data_transaction_supp_100.itertuples(index=False, name=None)), dtype=dtype)

# Créer un fichier HDF5 et y stocker les deux tables
with h5py.File("sales_data.h5", "w") as hd5_file:
    hd5_file.create_dataset('sales_sample', data=normal_data_array)
    hd5_file.create_dataset('sales_high_transaction', data=data_transaction_supp_100_array)

with h5py.File('sales_data.h5', 'r') as hdf:
    # Lire les données du table 'sales_sample'
    print(hdf['sales_sample'][0:5])